In [15]:
!pip install soundfile

In [16]:
import gradio as gr
from llama_index.llms.ollama import Ollama
from openai import OpenAI
from pathlib import Path
import os

In [20]:


# Initialize the Ollama LLaMA model
llm = Ollama(model="llama3.2:latest", request_timeout=60)

# Initialize OpenAI client for TTS
client = OpenAI()

def chat(input_text):
    # Generate response using LLaMA model
    response = llm.complete(input_text)
    response_text = response.text
    
    # Convert the response text to speech using OpenAI's TTS API
    audio_file_path = generate_speech(response_text)
    
    # Return the response text and the audio file path for Gradio
    return response_text, audio_file_path

def generate_speech(text):
    # Generate speech using OpenAI's TTS API without streaming response
    speech = client.audio.speech.create(
        model="tts-1",  # You can choose "tts-1" or "tts-1-hd"
        voice="alloy",  # Choose the voice you prefer ("alloy", "echo", etc.)
        input=text,
        response_format="mp3"  # Specify the response format (e.g., mp3)
    )

    # Get the binary response content (audio data)
    audio_data = speech.content  # Access the 'content' attribute

    # Define the audio file path based on the current working directory
    audio_file_path = os.path.join(os.getcwd(), "response_audio.mp3")
    
    # Write the binary content to the audio file
    with open(audio_file_path, "wb") as f:
        f.write(audio_data)  # Write the raw binary data

    # Return the path to the saved audio file
    return audio_file_path

# Gradio interface
interface = gr.Interface(
    fn=chat, 
    inputs="text", 
    outputs=["text", "audio"],  # Output text and audio file
    title="LLaMA 3 Chatbot with OpenAI TTS",
    description="Chat with a LLaMA 3-based model and hear the response via OpenAI's TTS!"
)

interface.launch(share=True)

Running on local URL:  http://127.0.0.1:7869
Running on public URL: https://6f820ede1f6106afb0.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
